In [1]:
import pickle, pathlib, os
import pandas as pd

In [2]:
file_name = 'TRXBTC_1h_EMA25.bin'
home_path = str(pathlib.Path.home())
data_path = os.path.join(home_path, file_name)

df = pickle.load(open(data_path, 'rb'))

In [5]:
pd.set_option('precision', 10) # output 10 decimal places

# copy value from df
ema_n_0 = df.iloc[-1].at['ema-25'] # latest ema value
ema_n_1 = df.iloc[-2].at['ema-25']  # last ema value

# trade_factor > 0 = price is rising / bullish trend
# trade_factor < 0 = price is falling / bearish trend
trade_factor = (ema_n_0 / ema_n_1) -1

In [8]:
# Validation Stage 2

n_cnt = 0


buy_price = 0.0
sell_price = 0.0

buy_factor = 0.004
sell_factor = -0.002


def validate(_buy_factor, _sell_factor):
    
    b_bought = False
    
    profit = 0.0
    
    for index, row in df[1:].iterrows(): # iterate over rows
        # start at second row and stop at penultimate row

        ema_n_1 = df.iloc[index][['ema-25']].values[0] # latest ema value
        ema_n_2 = df.iloc[index-1][['ema-25']].values[0] # latest ema value
        trade_factor = (ema_n_1 / ema_n_2) -1

        if trade_factor > _buy_factor and not b_bought: # ema-25 rises
            buy_price = df.iloc[index][['close']].values[0]
            b_bought = True

        elif trade_factor < _sell_factor and b_bought: # ema-25 is falling
            sell_price = df.iloc[index][['close']].values[0]
            b_bought = False

            profit += (sell_price / buy_price) -1

            
    return profit



            
profit = validate(buy_factor, sell_factor)

print("Profit so far: {:.2f}".format(profit))

Profit so far: -0.23


In [9]:
trading_factors = []

#iterate over buy factor from 0.001 to 0.009
for buy_factor in range(1, 9, 1):  
    buy_factor = buy_factor * 10**(-3)
    
    #iterate over buy factor from -0.001 to -0.009
    for sell_factor in range(1, 9, 1):      
        sell_factor = sell_factor * -10**(-3) 
        
        profit = validate(buy_factor, sell_factor)
        
        trading_factors.append((buy_factor, sell_factor, profit))

In [10]:
def select_profit(record):
    return record[2]

trading_factors.sort(key=select_profit, reverse=True)

In [11]:
trading_factors

[(0.002, -0.002, 0.03436033979967046),
 (0.002, -0.003, 0.014158319597650126),
 (0.006, -0.007, 0.0),
 (0.006, -0.008, 0.0),
 (0.007, -0.007, 0.0),
 (0.007, -0.008, 0.0),
 (0.008, -0.007, 0.0),
 (0.008, -0.008, 0.0),
 (0.001, -0.006, -0.00834652156004101),
 (0.002, -0.001, -0.031875007204029915),
 (0.002, -0.004, -0.03634673090740048),
 (0.002, -0.005, -0.03634673090740048),
 (0.002, -0.006, -0.03634673090740048),
 (0.001, -0.002, -0.04735133105695555),
 (0.006, -0.001, -0.07200000000000017),
 (0.007, -0.001, -0.07200000000000017),
 (0.008, -0.001, -0.07200000000000017),
 (0.006, -0.002, -0.08799999999999997),
 (0.007, -0.002, -0.08799999999999997),
 (0.008, -0.002, -0.08799999999999997),
 (0.006, -0.003, -0.10399999999999998),
 (0.007, -0.003, -0.10399999999999998),
 (0.008, -0.003, -0.10399999999999998),
 (0.001, -0.007, -0.11715481171548114),
 (0.001, -0.008, -0.11715481171548114),
 (0.002, -0.007, -0.11715481171548114),
 (0.002, -0.008, -0.11715481171548114),
 (0.001, -0.003, -0.12